In [1]:
!pip install contextily
!pip install geopandas
!pip install shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 74.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
import contextily as ctx
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import os
from datetime import datetime

In [3]:
file_paths = {
    2020: '311_2020.csv',
    2021: '311_2021.csv',
    2022: '311_2022.csv',
    2023: '311_2023.csv',
    2024: '311_2024.csv'
}

In [4]:
all_data = pd.DataFrame()

for year, file_path in file_paths.items():
    print(f"\nLoading data for {year}...")
    try:
        data = pd.read_csv(file_path)
        print(f"Successfully loaded {file_path} with {len(data)} records")

        # Filter for records with valid coordinates
        valid_data = data.dropna(subset=['latitude', 'longitude'])
        print(f"Found {len(valid_data)} records with valid coordinates")

        # Add year column
        valid_data['Year'] = year

        # Append to combined dataframe
        all_data = pd.concat([all_data, valid_data], ignore_index=True)

    except Exception as e:
        print(f"Error processing file for {year}: {e}")

print(f"\nTotal records with coordinates: {len(all_data)}")


Loading data for 2020...


<ipython-input-4-c5a786eb810b>:6: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


Successfully loaded 311_2020.csv with 251222 records
Found 249132 records with valid coordinates

Loading data for 2021...


<ipython-input-4-c5a786eb810b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['Year'] = year
<ipython-input-4-c5a786eb810b>:6: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


Successfully loaded 311_2021.csv with 273784 records
Found 271846 records with valid coordinates


...

Successfully loaded 311_2023.csv with 266438 records
Found 264309 records with valid coordinates


<ipython-input-4-c5a786eb810b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['Year'] = year



Loading data for 2024...
Successfully loaded 311_2024.csv with 282836 records
Found 280678 records with valid coordinates


<ipython-input-4-c5a786eb810b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['Year'] = year



Total records with coordinates: 1340909


In [5]:
# Check if we have data to process
if len(all_data) == 0:
    print("No data with valid coordinates found. Exiting.")
    exit()

# Extract coordinates for clustering
coordinates = all_data[['latitude', 'longitude']].values

# Standardize the coordinates
scaler = StandardScaler()
scaled_coordinates = scaler.fit_transform(coordinates)

# Determine optimal number of clusters using the Elbow Method
inertia = []
k_range = range(1, 11)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(scaled_coordinates)
    inertia.append(kmeans.inertia_)

# Plot the Elbow Method results
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertia, 'bo-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.grid(True)
plt.savefig('elbow_method.png')
plt.close()

# Choose optimal k (you can adjust this based on the elbow plot)
optimal_k = 5  # This should be adjusted after viewing the elbow plot
print(f"\nUsing {optimal_k} clusters based on the elbow method")

# Perform K-means clustering
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
all_data['cluster'] = kmeans.fit_predict(scaled_coordinates)

# Get cluster centers
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
cluster_info = pd.DataFrame(
    cluster_centers,
    columns=['center_latitude', 'center_longitude']
)
cluster_info['cluster'] = cluster_info.index
cluster_info['count'] = all_data.groupby('cluster').size().values

print("\nCluster information:")
print(cluster_info)

# Create a GeoDataFrame for visualization
geometry = [Point(xy) for xy in zip(all_data['longitude'], all_data['latitude'])]
gdf = gpd.GeoDataFrame(all_data, geometry=geometry, crs="EPSG:4326")

# Convert to web mercator for basemap
gdf_web = gdf.to_crs(epsg=3857)

# Create cluster centers GeoDataFrame
center_geometry = [Point(xy) for xy in zip(cluster_info['center_longitude'], cluster_info['center_latitude'])]
center_gdf = gpd.GeoDataFrame(cluster_info, geometry=center_geometry, crs="EPSG:4326")
center_gdf_web = center_gdf.to_crs(epsg=3857)

# Visualize the clusters
fig, ax = plt.subplots(figsize=(15, 12))

# Plot points colored by cluster
gdf_web.plot(
    column='cluster',
    ax=ax,
    alpha=0.6,
    markersize=5,
    categorical=True,
    legend=True,
    legend_kwds={'title': 'Clusters'}
)

# Plot cluster centers
center_gdf_web.plot(
    ax=ax,
    markersize=100,
    marker='*',
    color='red',
    edgecolor='black',
    label='Cluster Centers'
)

# Add basemap
ctx.add_basemap(ax, crs=gdf_web.crs.to_string(), source=ctx.providers.CartoDB.Positron)

# Add title and legend
plt.title('Spatial Clusters of 311 Reports (K-means)', fontsize=16)
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig('kmeans_clusters.png')
plt.close()

# Analyze cluster characteristics
print("\nAnalyzing cluster characteristics...")
cluster_analysis = all_data.groupby('cluster').agg({
    'latitude': ['mean', 'std'],
    'longitude': ['mean', 'std'],
    'Year': ['min', 'max', 'mean'],
    'subject': lambda x: x.value_counts().index[0] if len(x) > 0 else None,
    'case_title': lambda x: x.value_counts().index[0] if len(x) > 0 else None
}).reset_index()

print("\nCluster characteristics:")
print(cluster_analysis)

# Create a heatmap for each cluster
fig, axes = plt.subplots(1, optimal_k, figsize=(20, 6))
if optimal_k == 1:
    axes = [axes]  # Make axes iterable if only one cluster

for i in range(optimal_k):
    cluster_data = gdf_web[gdf_web['cluster'] == i]

    if len(cluster_data) > 0:
        ax = axes[i]
        # Create heatmap
        heatmap = ax.hexbin(
            cluster_data.geometry.x,
            cluster_data.geometry.y,
            gridsize=50,
            cmap='hot_r',
            alpha=0.7,
            mincnt=1
        )

        # Add basemap
        ctx.add_basemap(ax, crs=cluster_data.crs.to_string(), source=ctx.providers.CartoDB.Positron)

        # Add title
        ax.set_title(f'Cluster {i} (n={len(cluster_data)})')

        # Add colorbar
        plt.colorbar(heatmap, ax=ax, label='Density')
    else:
        axes[i].text(0.5, 0.5, f'No data for Cluster {i}',
                     horizontalalignment='center', verticalalignment='center')
        axes[i].set_axis_off()

plt.suptitle('Density Heatmaps by Cluster', fontsize=16)
plt.tight_layout()
plt.savefig('cluster_heatmaps.png')
plt.close()

# Temporal analysis of clusters
plt.figure(figsize=(12, 8))
for i in range(optimal_k):
    cluster_data = all_data[all_data['cluster'] == i]
    yearly_counts = cluster_data.groupby('Year').size()
    plt.plot(yearly_counts.index, yearly_counts.values, marker='o', linewidth=2, label=f'Cluster {i}')

plt.title('Yearly Distribution by Cluster')
plt.xlabel('Year')
plt.ylabel('Number of Reports')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('cluster_yearly_distribution.png')
plt.close()

print("\nClustering analysis complete. Output files saved.")



Using 5 clusters based on the elbow method

Cluster information:
   center_latitude  center_longitude  cluster   count
0        42.342365        -71.131002        0  177922
1       -71.112562         42.308522        1       4
2        42.306206        -71.074398        2  358059
3        42.353612        -71.058229        3  623024
4        42.276176        -71.129141        4  181900

Analyzing cluster characteristics...

Cluster characteristics:
  cluster   latitude            longitude            Year                     \
                mean       std       mean       std   min   max         mean   
0       0  42.342468  0.013239 -71.131190  0.021960  2020  2024  2022.128213   
1       1 -71.112562  0.042495  42.308522  0.045800  2020  2020  2020.000000   
2       2  42.305610  0.016075 -71.074559  0.015258  2020  2024  2021.969983   
3       3  42.353398  0.016248 -71.058430  0.018955  2020  2024  2022.084851   
4       4  42.276138  0.016749 -71.129477  0.019518  2020  2024  2